Notes on Various Topics for ARE212
==================================

**Author:** Ethan Ligon



## Kernel Density Estimation:PROPERTIES:



### The Wiggly Distribution



We address the following problem.  There exists a random variable
with a density $f$.  We call this distribution the &ldquo;wiggly&rdquo;
distribution, and use the magic of `scipy.stats` to define it as follows:



In [1]:
from scipy.stats import rv_continuous
import numpy as np

class wiggly_gen(rv_continuous):

    """Wiggly distribution"""

    def _pdf(self, x):
        d = self.b - self.a
        return (np.sin(x*6*2*np.pi/d) + 1)/(self.b - self.a)

wiggly = wiggly_gen(a=0.,b=2*np.pi,name='wiggly')

x = wiggly()

f = x.pdf  # Name of true pdf, for pedagogical convenience

Plot the pdf:



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def plot_pdf(x,npts=1000,ax=None):

    if ax is None:
        fig,ax = plt.subplots()

    V = np.linspace(x.a,x.b,npts)
    ax.plot(V,[x.pdf(v) for v in V])

    return ax

ax = plot_pdf(x)

### Sample



Now, suppose we don&rsquo;t know the distribution of the random variable
$x$, but we can draw a sample of realizations, which we&rsquo;ll put in a `pandas.Series`.



In [1]:
import pandas as pd

n = 1000 # Sample size

S = pd.Series(x.rvs(size=n))

### Estimation



How can we use this random sample to estimate the density?
One simple idea is to look at the histogram (scaled to sum to one):



In [1]:
S.hist(bins=int(np.sqrt(len(S))),density=True)

This captures some of the important features of the distribution, but
let&rsquo;s see if we can do better by using kernel density tools.

Start by defining a kernel:



In [1]:
sqrt3 = np.sqrt(3)  # Avoid repeated evaluation of this for speed...

k = lambda u: (np.abs(u) < sqrt3)/(2*sqrt3)  # Rectangular kernel

# k = lambda u: np.exp(-(u**2)/2)/np.sqrt(2*np.pi) # Gaussian kernel

Now define the kernel estimator:



In [1]:
def kernel_estimator(X,h):
    """
    Use data X to estimate a density, using bandwidth h.
    """
    return lambda x: k((X-x)/h).mean()/h

We already have a random sample: let&rsquo;s try using it to estimate $f$:



In [1]:
fhat = kernel_estimator(S,0.05) 

fhat(1)  # Try to evaluate at a point

Now graph our estimate



In [1]:
ax = plot_pdf(x) # "True" distribution

V = np.linspace(0,2*np.pi,1000)

ax.plot(V,[fhat(x) for x in V])

Play with other bandwidths, and compare with &ldquo;truth&rdquo;



In [1]:
ax = plot_pdf(x) # "True" distribution

fhat = kernel_estimator(S,0.25) 

ax.plot(V,[fhat(x) for x in V])

Consider the Silverman rule of thumb:



In [1]:
h_silverman = S.std()*S.count()**(-1/5)*1.06

ax = plot_pdf(x) # "True" distribution

fhat = kernel_estimator(S,h_silverman) 

ax.plot(V,[fhat(x) for x in V])

### Bias



Hansen (Probability, &sect; 17.4) develops an expression for the bias of
the  kernel density estimator:
$$
  \mbox{Bias}(x) = \int k(u)\left(f(x+hu)-f(x)\right)du.
$$
Thus, bias depends only on the kernel, the bandwidth, and the
(unknown!) density $f$.

Here we use an &ldquo;oracle&rdquo; estimator to estimate the bias, in which we
make use of the fact that (for the experiment in this notebook) we in
fact *do* do know $f$.



#### Oracle Bias Estimator



In [1]:
from scipy.integrate import quad # General purpose integration

integrand = lambda u,x,h: k(u)*(f(x + h*u) - f(x))

# bias will return a pair (integral,absolute error tolerance),
# where the latter is a promise that the absolute error 
# from numerical integration is smaller than the reported tolerance.
bias = lambda x,h: quad(lambda u: integrand(u,x,h), # Hold x & h fixed in integral
                        a=0, b=2*np.pi)   # Limits of integration

Try evaluating this:



In [1]:
bias(x=1,h=0.1)

Try plotting!



In [1]:
ax = plot_pdf(x)
h = 0.1
ax.plot(V,[bias(v,h=h) for v in V])

Compare with actual estimate:



In [1]:
ax = plot_pdf(x)

h = 0.1

fhat = kernel_estimator(S,h=h) 

ax.plot(V,[bias(v,h=h) for v in V])

ax.plot(V,[fhat(v) for v in V])

### Variance



The finite-sample *variance* of the kernel estimator (so now sampling
variation matters) can be computed as just the variance of the sample
&ldquo;smooths&rdquo; $k(x_i - x)/h$, so



In [1]:
def Vhat(X,h):
    """
    Use data X to estimate the variance of $\hat{f}$ as a function of $x$.
    """
    n = X.count()
    return lambda x: k((X-x)/h).var()/(n*h**2)

Try evaluating this:



In [1]:
vhat = Vhat(S,h=0.1)

Try plotting!



In [1]:
plt.plot(V,[vhat(v) for v in V])

## Kernel Regression:PROPERTIES:



### Conditional expectations



We address the following problem.  There exists a random variable $y$
  with conditional expectation $E(y|x)$  which is continuous in
  $x$ (further, if $x$ is random then the density of $x$, $f(x)>0$).
  For example (play!):



In [1]:
import numpy as np

def m(x): return np.sin(2*x)/(1+x)

Plot the conditional expectation:



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

V = np.linspace(1e-4,2*np.pi,1000)

ax = plt.plot(V,m(V))

### Sample



Now, suppose we don&rsquo;t know $m$ or the conditional distribution of $y$,
but we can draw a sample of realizations, which we&rsquo;ll put in a `pandas.Series`.



In [1]:
import pandas as pd
from scipy.stats import distributions as iid

X = iid.uniform(V.min(),V.max())  # Handy to have bounded support, but not necessary

U = iid.norm(scale=0.1) # Scale=std. dev.  Play!

n = 1000 # Sample size

x = pd.Series(X.rvs(size=n))

# Series are imperial when they can be:
y = m(x) + U.rvs(size=n)  

type(y)

Now let&rsquo;s take an informal look at the data:



In [1]:
plt.scatter(x,y)

How can we use this random sample to estimate $m$?  The scatterplot
suggests that some kind of local smoothing might help.



### Estimator



Let&rsquo;s go ahead and define the kernel regression estimator, following
the same general logic as in our exploration of kernel densities.

Start by defining a kernel:



In [1]:
sqrt3 = np.sqrt(3)  # Avoid repeated evaluation of this for speed...

k = lambda u: (np.abs(u) < sqrt3)/(2*sqrt3)  # Rectangular kernel

# k = lambda u: np.exp(-(u**2)/2)/np.sqrt(2*np.pi) # Gaussian kernel

Now define the kernel estimator:



In [1]:
def kernel_regression(X,y,h):
    """
    Use data (X,y) to estimate E(y|x), using bandwidth h.
    """
    def mhat(x):
        S = k((X-x)/h) # "Smooths"

        return S.dot(y)/S.sum()

    return mhat

We already have a random sample: let&rsquo;s try using it to estimate $m$:



In [1]:
mhat = kernel_regression(x,y,h=0.3) 

mhat(1)  # Try to evaluate at a point

Now graph our estimate



In [1]:
fig,ax = plt.subplots()

ax.plot(V,[m(v) for v in V])  # "Truth"
ax.plot(V,[mhat(v) for v in V]) # Estimate

### Leave-one out cross-validation



We talked about computing estimates $m_{-i}$ to estimate the IMSE.
   Doing this directly would require $n$ separate estimations.  This
   is expensive!

A little algebra suggests an alternative, involving the &ldquo;Gram&rdquo; matrix.



In [1]:
def leave_out_residuals(X,y):
    """
    Use data (X_{-j},y_{-j}) to estimate E(y|x), using bandwidth h.
    """
    
    def Gram_matrix(X,h):
        """
        Define kernel-data matrix [ k((x_i-x_j)/h) ]
        """
        v = np.array(X).reshape((-1,1)) # Column vector

        return k((v-v.T)/h) 

    def ecv(h):
        """
        Leave-out residuals as fn of bandwidth h.
        """
        G = Gram_matrix(X,h)
        G0 = G - np.diag(np.diag(G)) # Zero out diagonals

        e = y - (G0@y)/np.sum(G0,axis=1)

        return e

    return ecv

Consider the &ldquo;leave-one-out&rdquo; or &ldquo;Cross-Validation&rdquo; residuals:



In [1]:
ecv = leave_out_residuals(x,y)

ax = ecv(h=0.1).hist()

Define an estimator of the IMSE as a function of the bandwidth $h$:



In [1]:
def imse_cv(h):
    return np.mean(ecv(h)**2)

imse_cv(.5)

Try plotting esimated IMSE as a function of bandwidth:



In [1]:
H = np.linspace(1e-3,.5,20)

plt.plot(H,[imse_cv(h) for h in H])

Actually find the minimum:



In [1]:
from scipy.optimize import minimize_scalar

soltn = minimize_scalar(imse_cv,bounds=[1e-2,1])

if soltn.success:
    hstar = soltn.x
    print(soltn)

Now, estimate using estimated optimal bandwidth:



In [1]:
mhat = kernel_regression(x,y,hstar)

fig,ax = plt.subplots()

ax.plot(V,[m(v) for v in V])  # "Truth"
ax.plot(V,[mhat(v) for v in V]) # Estimate

## Quantities & Prices (Wright 1934)



Consider a data generating process for $(q,p)$ based
on Goldberger (1972), who in turn is describing the work of Sewall
Wright (1934).  The demand-supply system is
$$
   q_D = \alpha p + u\qquad q_S = \beta p + v\qquad q_D = q_S,
$$
where $(u,v)$ are unobserved shocks to demand and supply,
respectively.



In [1]:
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid

# Structural parameters;
(α,β) = (-1,2)     
σ = {'u':1/2,'v':1/3}
μ = {'u':2,'v':-1}

# u,v assumed independent
u = iid.norm(loc=μ['u'], scale=σ['u'])  # Demand shocks
v = iid.norm(loc=μ['v'], scale=σ['v'])  # Supply shocks

# Reduced form coefficients
π = [[-β/(α - β), -1/(α - β)],
     [ α/(α - β), 1/(α - β)]]

# Generate N realizations of system
# Outcomes Y have columns (q,p)
N = 10

# Arrange shocks into an Nx2 matrix
U = np.c_[u.rvs(N), v.rvs(N)]

# Matrix product gives [q,p]; label by putting into df
df = pd.DataFrame(U@π,columns=['q','p'])
Udf = pd.DataFrame(U,columns=['u','v']) # For future reference

We can interrogate these data:



In [1]:
df

And compute the linear correlation&#x2026;



In [1]:
df.corr()

Or more generally the covariance matrix:



In [1]:
C=df.cov()
C

From which we can calculate the linear regression coefficient
of $p = a + bq + e$:



In [1]:
C.loc['p','q']/C.loc['q','q']

And learn about the probability density&#x2026;



In [1]:
from scipy import stats
import numpy as np

# Estimate joint density of (q,p)
pdf = stats.gaussian_kde(df.T).pdf 

ax = df.plot.scatter(x='q',y='p')

v = ax.axis()
Q = np.mgrid[v[0]:v[1]:100j].tolist()
P = np.mgrid[v[2]:v[3]:100j].tolist()

_ = ax.contour(Q,P,np.array([[pdf((q,p))[0] for p in P] for q in Q]))

#### Counterfactual Demand & Supply Schedules



What are the actual *counterfactual* demand and
supply schedules?  This is the kind of thing that Frisch described as
&ldquo;hypothetical experiments.&rdquo;  The schedules respond to shocks $u$ and $v$, respectively,
yielding



In [1]:
qmax = df['q'].max()
qmin = df['q'].min()

Q = pd.DataFrame({'min':np.maximum(0,df['q']-0.3*(qmax-qmin)),
                  'max':np.minimum(qmax*1.2,df['q']+0.3*(qmax-qmin)),
                  'miss':-1})

# Inverse counterfactual demand & supply (for plotting)
D = Q.add(-Udf['u'],axis=0)/α  
S = Q.add(-Udf['v'],axis=0)/β

counterfactual=pd.DataFrame({'S':S.stack(),
                             'D':D.stack(),
                             'Q':Q.stack()})

counterfactual=counterfactual.replace(-1,np.nan)

_ = counterfactual.plot(x='Q')

#### Controlling Price



Consider the question: what would expected demand be if we *fixed*
    the price at $p_0$?  Expected supply?



#### Average Causal Effect of a Change in Price



What would expected demand be if we *observed* that the price was $p_0$?



#### Price Change *Ceteris Paribus*



Suppose we *observe* prices and quantities $(p_0,q_0)$.  How *would*
    we expect the quantity demanded to change if prices were instead
    fixed at $p_1$, *ceteris paribus*?



## Instrumental Variables in Canonical Demand & Supply Model



### Data-Generating Process



In [1]:
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid

# Unobservable component of supply shock z
# Can have any distribution one pleases
w = iid.beta(1,2,loc=-iid.beta(1,2).mean()) # Centered for convenience

# Structural parameters;
(alpha,beta) = (-1,2)     
sigma = {'u':1/2,'v':1/3}
mu = {'u':2,'v':-1}

# u,v assumed independent
u = iid.norm(loc=mu['u'], scale=sigma['u'])  # Demand shocks
v = iid.norm(loc=mu['v'], scale=sigma['v'])  # Supply shocks

# Reduced form coefficients
pi = [[-beta/(alpha - beta), -1/(alpha - beta)],
     [ alpha/(alpha - beta), 1/(alpha - beta)]]

# Generate N realizations of system
# Outcomes have columns (p,q,z)
def wright_dgp(N):
    """
    Generate data consistent with Wright (1934) hog demand and supply.

    Returns a pandas dataframe with N observations on (p,q,z), where
    z is understood to be a supply shock.
    """
    
    # Arrange shocks into an Nx2 matrix
    U = np.c_[u.rvs(N), v.rvs(N)]

    # Matrix product gives [q,p]; label by putting into df
    df = pd.DataFrame(U@pi,columns=['q','p'])

    Udf = pd.DataFrame(U,columns=['u','v']) # For future reference

    # Relate v and z (need not be linear)
    unobserved_shock = w.rvs(N)/10
    df['z'] = (1-unobserved_shock)*np.exp(4*Udf['v'] - unobserved_shock)
    df['Constant'] = 1

    # Include a constant term in both X & Z
    return df[['q']],df[['Constant','p']],df[['Constant','z']]

### Estimation



Let&rsquo;s write some code to estimate the parameters of the regression
   model using the estimator devised above (the &ldquo;simple IV estimator&rdquo;):



In [1]:
import numpy as np

def draw_b(N,dgp):
    """
    Generate a random variate $b$ from a sample of $N$ draws from the Wright (1934) DGP.
    """
    y,X,Z =  dgp(N)

    return np.linalg.solve(Z.T@X,Z.T@y) # Solve normal eqs

b = draw_b(10000,wright_dgp)

print(b)

### Inference



Now consider the point that the estimator $b$ is a random variable.
 Under the assumptions of the *model* a Central Limit Theorem applies,
 so it&rsquo;s asymptotically normal.  But in any finite sample the just
 identified linear IV estimator can be feisty.  Let&rsquo;s explore using a
 little Monte Carlo experiment.  Let&rsquo;s begin by constructing a
 slightly more transparent data-generating process, in which $Z$ and
 $X$ have a linear relationship:



In [1]:
from scipy.stats import distributions as iid

def linear_dgp(N,beta,gamma,pi,sigma_u,sigma_v):
    u = iid.norm(scale=sigma_u).rvs(N)
    v = iid.norm(scale=sigma_v).rvs(N)
    Z = iid.norm().rvs(N)

    X = Z*pi + v
    y = X*beta + u

    df = pd.DataFrame({'y':y,'x':X,'z':Z,'Constant':1})

    return df[['y']],df[['Constant','x']],df[['Constant','z']]

The next bit of code *repeatedly* draws new random samples and
  calculates $b$ from them; we then construct a histogram of the
  resulting estimates.



In [1]:
from matplotlib import pyplot as plt

B = pd.DataFrame([draw_b(100,lambda N: linear_dgp(N,1,0,.01,1,1))[1] for i in range(1000)])
B.hist(bins=int(np.ceil(np.sqrt(B.shape[0]))))

## Finite Sample Properties of Linear GMM



### Introduction



GMM provides a generalized way to think about instrumental
   variables estimators, and we also have evidence that the finite
   sample properties of these estimators may be poor.  Here we&rsquo;ll
   construct a simple Monte Carlo framework within which to evaluate
   the finite-sample behavior of GMM linear IV estimators.



### Asymptotic Variance of GMM estimator



If we have $\mbox{E}g_j(\beta)g_j(\beta)^\top=\Omega$ and
   $\mbox{E}\frac{\partial g_j}{\partial b^\top}(\beta)=Q$ then we&rsquo;ve
   seen that the asymptotic variance of the optimally weighted GMM
   estimator is
   $$
       V_b = \left(Q^\top\Omega^{-1}Q\right)^{-1}.
   $$



### Data Generating Process



We consider the linear IV model

\begin{align*}
   y &= X\beta + u\\
   \mbox{E}Z^\top u &= 0
\end{align*}

Thus, we need to describe processes that generate $(X,Z,u)$.

The following code block defines the important parameters governing
the DGP; this is the &ldquo;TRUTH&rdquo; we&rsquo;re designing tools to reveal.



In [1]:
import numpy as np
from numpy.linalg import inv

## Play with us!
beta = 1     # "Coefficient of interest"
gamma = 1    # Governs effect of u on X
sigma_u = 1  # Note assumption of homoskedasticity
## Play with us!

# Let Z have order ell, and X order 1, with Var([X,Z]|u)=VXZ

ell = 4 # Play with me too!

# Arbitrary (but deterministic) choice for VXZ = [VX Cov(X,Z);
#                                                 Cov(Z,X) VZ]
# Pinned down by choice of a matrix A...
A = np.sqrt(1/np.arange(1,(ell+1)**2+1)).reshape((ell+1,ell+1)) 

## Below here we're less playful.

# Now Var([X,Z]|u) is constructed so guaranteed pos. def.
VXZ = A.T@A 

Q = -VXZ[1:,[0]]  # -EZX', or generally Edgj/db'

# Gimme some truth:
truth = (beta,gamma,sigma_u,VXZ)

## But play with Omega if you want to introduce heteroskedascity
Omega = (sigma_u**2)*VXZ[1:,1:] # E(Zu)(u'Z')

# Asymptotic variance of optimally weighted GMM estimator:
print(inv(Q.T@inv(Omega)@Q))

Now code to generate $N$ realizations of $(y,X,Z)$ given some &ldquo;truth&rdquo;
`(beta,gamma,sigma_u,VXZ)`:



In [1]:
from scipy.stats import distributions as iid

def dgp(N,beta,gamma,sigma_u,VXZ):
    """Generate a tuple of (y,X,Z).

    Satisfies model:
        y = X@beta + u
        E Z'u = 0
        Var(u) = sigma^2
        Cov(X,u) = gamma*sigma_u^2
        Var([X,Z}|u) = VXZ
        u,X,Z mean zero, Gaussian

    Each element of the tuple is an array of N observations.

    Inputs include
    - beta :: the coefficient of interest
    - gamma :: linear effect of disturbance on X
    - sigma_u :: Variance of disturbance
    - VXZ :: Cov([X,Z|u])
    """
    
    u = iid.norm.rvs(size=(N,1))*sigma_u

    # "Square root" of VXZ via eigendecomposition
    lbda,v = np.linalg.eig(VXZ)
    SXZ = v@np.diag(np.sqrt(lbda))

    # Generate normal random variates [X*,Z]
    XZ = iid.norm.rvs(size=(N,VXZ.shape[0]))@SXZ.T

    # But X is endogenous...
    X = XZ[:,[0]] + gamma*u
    Z = XZ[:,1:]

    # Calculate y
    y = X*beta + u

    return y,X,Z

Check on DGP:



In [1]:
N = 1000

data = dgp(N,*truth)

y,X,Z = data # Unpack tuple to check on things

# Check that we've computed things correctly:
print("True Var([X,Z]|u):")
print(VXZ,end="\n\n")

print("Estimated (unconditional) sample covariance matrix minus true conditional:")
print(np.cov(np.c_[X,Z].T,ddof=0) - VXZ)

### Estimation



Now that we have a data-generating process we proceed under
   the conceit that we can observe samples generated by this process,
   but otherwise temporarily &ldquo;forget&rdquo; the properties of the DGP, and use the
   generated data to try to reconstruct aspects of the DGP.

In our example, we consider using the optimally weighted linear IV
estimator, and define a function which computes observation-level
deviations from expectations for this model. To estimate a different
model this is the function we&rsquo;d want to re-define `gj`.



In [1]:
import numpy as np
from numpy.linalg import inv

def gj(b,y,X,Z):
    """Observations of g_j(b).

    This defines the deviations from the predictions of our model; i.e.,
    e_j = Z_ju_j, where EZ_ju_j=0.

    Can replace this function to testimate a different model.
    """
    return Z*(y - X*b)

#### Construct sample moments



Begin by defining a function to construct the sample moments given
    the data and a parameter estimate $b$:



In [1]:
def gN(b,data):
    """Averages of g_j(b).

    This is generic for data, to be passed to gj.
    """
    e = gj(b,*data)

    # Check to see more obs. than moments.
    assert e.shape[0] > e.shape[1]
    
    return e.mean(axis=0)

#### Define estimator of Egg&rsquo;



Next we define a function to compute covariance matrix of moments.
Re-centering can be important in finite samples, even if irrelevant in
the limit.  Since we have $\mbox{E}g_j(\beta)=0$ under the null we may
as well use this information when constructing our weighting matrix.



In [1]:
def Omegahat(b,data):
    e = gj(b,*data)

    # Recenter! We have Eu=0 under null.
    # Important to use this information.
    e = e - e.mean(axis=0) 
    
    return e.T@e/e.shape[0]

Check construction of weighting matrix for our data at true parameter
$\beta$.  Note that we&rsquo;re &ldquo;cheating&rdquo; here, since our construction of
`Winv` relied on knowledge of the true parameter $\beta$ (this is
sometimes called an &ldquo;Oracle estimator&rdquo;).  We&rsquo;ll play fair a bit later.



In [1]:
Winv = Omegahat(beta,data) 
print(Winv)

Define the criterion function given a weighting matrix $W$:



In [1]:
def J(b,W,data):

    m = gN(b,data) # Sample moments @ b
    N = data[0].shape[0]

    return N*m.T@W@m # Scale by sample size

Next, check construction of criterion given our data.  We want
something that looks nice and quadratic, at least in the neighborhood
of $\beta$.  Note that comparing the criterion to the critical values
of the $\chi^2$ statistic gives us an alternative way to construct
confidence intervals.



In [1]:
from matplotlib import pyplot as plt
%matplotlib inline

# Limiting distribution of criterion (under null)
limiting_J = iid.chi2(ell-1)

# Limiting SE of b
sigma_0 = lambda N: np.sqrt(inv(Q.T@inv(Winv)@Q)/N)[0][0] 

# Choose 8 sigma_0 neighborhood of "truth"
B = np.linspace(beta-4*sigma_0(N),beta+4*sigma_0(N),100)
W = inv(Winv)

_ = plt.plot(B,[J(b,W,data) for b in B.tolist()])
_ = plt.axhline(limiting_J.isf(0.05),color='r')

#### Two Step Estimator



We next implement the two-step GMM estimator.  If we needed to
    estimate more than a single scalar parameter we&rsquo;d need a different
    minimization routine.



In [1]:
from scipy.optimize import minimize_scalar

def two_step_gmm(data):

    # First step uses identity weighting matrix
    W1 = np.eye(gj(1,*data).shape[1])

    b1 = minimize_scalar(lambda b: J(b,W1,data)).x 

    # Construct 2nd step weighting matrix using
    # first step estimate of beta
    W2 = inv(Omegahat(b1,data))

    return minimize_scalar(lambda b: J(b,W2,data))

Now let&rsquo;s try it with an actual sample, just to see that things work:



In [1]:
soltn = two_step_gmm(data)

print("b=%f, J=%f, Critical J=%f" % (soltn.x,soltn.fun,limiting_J.isf(0.05)))

### Monte Carlo Experiment



Now our experiment begins.  We set our frequentist hats firmly on our
heads, and draw repeated samples of data, each generating a
corresponding estimate of beta.  Then the empirical distribution of
these samples tells us about the *finite* sample performance of our estimator.

We&rsquo;ll generate a sample of estimates of $b$ by drawing repeated
samples of size $N$:



In [1]:
N = 1000 # Sample size

D = 1000 # Monte Carlo draws

b_draws = []
J_draws = []
for d in range(D):
    soltn = two_step_gmm(dgp(N,*truth))
    b_draws.append(soltn.x)
    J_draws.append(soltn.fun)

_ = plt.hist(b_draws,bins=int(np.ceil(np.sqrt(N))))
_ = plt.axvline(beta,color='r')

### Distribution of Monte Carlo draws vs. Asymptotic distribution



Compare Monte Carlo standard errors with asymptotic approximation:



In [1]:
# Limiting distribution of estimator

limiting_b = iid.norm(scale=sigma_0(N))

print("Bootstrapped standard errors: %g" % np.std(b_draws))
print("Asymptotic approximation: %g" % sigma_0(N))
print("Critical value for J statistic: %g (5%%)" % limiting_J.isf(.05))

Now construct probability plot (bootstrapped $b$s vs. quantiles of
limiting distribution):



In [1]:
from scipy.stats import probplot

_ = probplot(b_draws,dist=limiting_b,fit=False,plot=plt)

Next, consider the a $p$-$p$ plot for $J$ statistics (recall these
should be distributed $\chi^2_{\ell-1}$).



In [1]:
def ppplot(data,dist):
    data = np.array(data)

    # Theoretical CDF, evaluated at points of data
    P = [dist.cdf(x) for x in data.tolist()]

    # Empirical CDF, evaluated at points of data
    Phat = [(data<x).mean() for x in data.tolist()]

    fig, ax = plt.subplots()
    
    ax.scatter(P,Phat)
    ax.plot([0,1],[0,1],color='r') # Plot 45
    ax.set_xlabel('Theoretical Distribution')
    ax.set_ylabel('Empirical Distribution')
    ax.set_title('p-p Plot')

    return ax
    
_ = ppplot(J_draws, limiting_J)

## Example use of GMM class



I&rsquo;ve turned the code we developed last week into a class.  Here&rsquo;s a
simple example which  estimates a Mincer wage regression using data
from Angrist-Krueger (1991):



In [1]:
import pandas as pd

ak = pd.read_stata('angrist-krueger91.dta')

y = ak.logwage
X = ak[['ageq','edu','married']]
X['Constant'] = 1

Next we define a function that has the property that it&rsquo;s expected
value at a true vector of parameters is equal to zero.  First, just a
simple Mincer wage regression:



In [1]:
def linear_regression(b,data):
    """
    Return matrix X.T*e (ell x N)
    """
    y,X=data
    e = y - X@b

    return (X.T*e).T

Import the GMM class



In [1]:
from gmm import GMM

GMM?

Now instantiate the GMM problem for this restriction, using the
Angrist-Krueger data.



In [1]:
mygmm = GMM(linear_regression,(y,X),4)

## GMM Estimation of Logit Model



### Introduction



Consider a &ldquo;logit&rdquo; regression; score function from MLE is:
  $$
     \frac{1}{N}\sum_j X_j\left(y_j - \frac{e^{X_j\beta}}{1+e^{X_j\beta}}\right) = 0.
  $$
  Let $u_j = y_j - \frac{e^{X_j\beta}}{1+e^{X_j\beta}}$; if we have some $Z$ such that
  $\mbox{E}(u|Z) = 0$ then we can construct further moment conditions
  $$
     \frac{1}{N}\sum_j Z_j\left(y_j - \frac{e^{X_j\beta}}{1+e^{X_j\beta}}\right) = 0.
  $$



### GMM Estimator



In [1]:
import numpy as np

# Some different options for moments...

def mle(b,data):
    """Observations of score for MLE estimator.

    Moment condition is E(y_j - p(x_jb))x_j = 0,
    where p(xb) = e^{xb}/(1+e^{xb})
    """
    # Orient so that data are matrices with N rows
    y,X,Z = data
    N = np.max(y.shape)
    y = y.reshape((N,-1))
    X = X.reshape((N,-1))
    Z = Z.reshape((N,-1))
    
    # Construct vector of identical parameter if b a scalar.
    if np.isscalar(b) or len(b)==1: b = np.array([b]*X.shape[1])
    b = np.array(b)
    k = np.max(b.shape)
    b = b.reshape((k,1))

    p = np.exp(X@b)  # This is actually the odds
    p = p/(1+p)      # This is probability y=1

    return (X*(y - p))

def nonlinear_iv(b,data):
    """Observations for restriction that Z
    orthogonal to score.

    Moment condition is E(Z_jy_j - Z_jexp(x_jb)) = 0
    """
    y,X,Z = data
    # Orient so that data are matrices with N rows
    N = np.max(y.shape)
    y = y.reshape((N,-1))
    X = X.reshape((N,-1))
    Z = Z.reshape((N,-1))

    # Construct vector of identical parameter if b a scalar.
    if np.isscalar(b) or len(b)==1: b = np.array([b]*X.shape[1])
    b = np.array(b)
    k = np.max(b.shape)
    b = b.reshape((k,1))

    p = np.exp(X@b)  # This is actually the odds
    p = p/(1+p)      # This is probability y=1

    return (Z*(y - p))

### Data Generating Process



In [1]:
from scipy.stats import distributions as iid

def dgp(N,beta,VXZ,gamma=1):
    """Generate a tuple of (y,X,Z).

    Satisfies model:
        Pr(y=1|X) = f(X@beta,gamma)
        u = y - f(X@beta,gamma)
        E(u|Z) = 0
        f(x,gamma) = (exp(x)/(1+exp(x)))**gamma
        Var([X,Z}) = VXZ
        X,Z mean zero, Gaussian

    Each element of the tuple is an array of N observations.
    When gamma=1 this reduces to the logit model

    Inputs include
    - beta :: Governs effect of X on probability y=1
    - gamma :: Governs curvature of function
    - VXZ :: Var([X,Z])
    """
    
    # "Square root" of VXZ via eigendecomposition
    lbda,v = np.linalg.eig(VXZ)
    SXZ = v@np.diag(np.sqrt(lbda))

    # Generate normal random variates [X*,Z]
    XZ = iid.norm.rvs(size=(N,VXZ.shape[0]))@SXZ.T

    # Add a constant to both X & Z
    X = np.c_[np.ones((N,1)),XZ[:,[0]]]
    Z = np.c_[np.ones((N,1)),XZ[:,1:]]

    # Calculate y
    pi = np.exp(X.dot(beta))
    pi = (pi/(1+pi))**gamma

    y = iid.bernoulli(pi).rvs()

    return y,X,Z

### The Truth (Mark I)



In this version of the truth the form of the distribution of $y$ is
known up to the parameter $\beta$; MLE takes advantage of this.

Choose some parameters to establish the &ldquo;truth&rdquo;:



In [1]:
import numpy as np
from numpy.linalg import inv

## Play with us!
beta = np.array([0,2])     # "Coefficient of interest"

## Play with us!

# Let Z have order ell, and X order 1, with Var([X,Z]|u)=VXZ

ell = 2 # Play with me too!

# Arbitrary (but deterministic) choice for VXZ
A = np.sqrt(1/np.arange(1,(ell+1)**2+1)).reshape((ell+1,ell+1)) 

## Below here we're less playful.

# Var([X,Z]|u) is constructed so that pos. def.
VXZ = A.T@A 

truth = (beta,VXZ)

### Monte Carlo Analysis of MLE via GMM



Here we take the score to be our moment condition; $Z$ doesn&rsquo;t appear,
so estimator is just identified.



In [1]:
import gmm # Just code defined last class

gmm.gj = mle  # Redefine gj function to use MLE score

data = dgp(1000,*truth)

bhat,Jhat = gmm.two_step_gmm(data,b_init=np.zeros((2,1)))

limiting_J = iid.chi2(0)

print("b=(%f,%f), J=%f, Critical J=%f" % (bhat[0],bhat[1],Jhat,limiting_J.isf(0.05)))

Now our experiment begins.  We set our frequentist hats firmly on our
heads, and draw repeated samples of data, each generating a
corresponding estimate of beta.  Then the empirical distribution of
these samples tells us about the *finite* sample performance of our estimator.

We&rsquo;ll generate a sample of estimates of $b$ by drawing repeated
samples of size $N$, until estimates of the covariance of our
estimates converge:



In [1]:
import gmm # Just module we discussed in class, but we're using it
           # procedurally instead of as a class.

N = 1000 # Sample size

tol = 1e-3

b_mle = []
J_mle = []
b_nliv = []
J_nliv = []

d=0
oldV = 0
newV = 1
b = np.zeros((2,1))
while d<30 or np.linalg.norm(oldV-newV) > tol:
    d += 1
    oldV = newV
    data = dgp(N,*truth)
    gmm.gj = mle
    b,J = gmm.two_step_gmm(data,b_init=b)
    b_mle.append(b)
    J_mle.append(J)

    gmm.gj = nonlinear_iv
    b,J = gmm.two_step_gmm(data,b_init=b)
    b_nliv.append(b)
    J_nliv.append(J)

    newV = np.var(b_nliv)

Now compare MLE & NLIV estimates:



In [1]:
from matplotlib import pyplot as plt

_ = plt.scatter(b_mle,b_nliv)

## Linear Non-Linear Functions



### Preface



Some modules we&rsquo;ll want:



In [1]:
import numpy as np
from numpy.linalg import norm
from scipy.stats import distributions as iid
import pandas as pd
import matplotlib.pyplot as plt

### Factories to generate basis functions



We can use a collection of functions as a *basis* with which to represent
an arbitrary function.



In [1]:
# Factory function for phi_k(x)
phi_factory = lambda c,s=1: lambda x: np.exp(-(1/(2*s))*norm(x-c)**2)  # RBF
# phi_factory = lambda c,s=1: lambda x: (x**c)/s  # Polynomial
# phi_factory = lambda c,s=1: lambda x: 1/(1+np.exp(c-x))  # logistic

# Also chose a domain over which we'll want to evaluate the unknown function
Domain = np.linspace(0,2*np.pi,100).tolist()

Now use  this factory to generate a set of $K$ basis functions for our
representation:



In [1]:
## Or
K=18

# Divide up domain
phis = {k:phi_factory(Domain[x]) for k,x in enumerate(range(1,len(Domain),len(Domain)//K))}

# Gram matrix
#phis = {k:phi_factory(X[k]) for k in range(K)}

phis[0] = lambda x: 1 # Constant function

Plot the basis functions:



In [1]:
for k in range(K):
    plt.plot(Domain,[phis[k](x) for x in Domain])

### Data generating function



Let $f_0(X) = \mbox{E}(y|X)$ be an arbitrary function.  
After specifying this function we define a data data-generating
function for $(X,y)$ satisfying $y=f(X) + u$ and $\mbox{E}(u|X)$.



In [1]:
f0 = lambda x: x*np.sin(x) # True function; make up your own!

def dgp(N,sigma_u):
    X = iid.uniform(loc=0,scale=2*np.pi).rvs(N).tolist()
    X.sort()

    u = iid.norm(scale=sigma_u)

    y = pd.Series([f0(x) + u.rvs(1)[0] for x in X])

    return X,y

N = 20
X,y = dgp(N,1)

Consider scatterplot:



In [1]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.scatter(X,y)

ax.plot(Domain,[f0(x) for x in Domain])

### Estimation



Now regression.  Suppose we don&rsquo;t know the true function `f0`, and
can only estimate using observed data.



In [1]:
TX = {}
for k in phis.keys():
    TX[k] = [phis[k](x) for x in X]

TX = pd.DataFrame(TX)

try: # If y isn't a DataFrame make it one
    y = pd.DataFrame({'y':y})
except ValueError: # Guess it is!
    pass

alpha = pd.DataFrame(np.linalg.lstsq(TX, y,rcond=None)[0],index=TX.columns)

# Check fit:
e = y.squeeze() - (TX@alpha).squeeze()
e.var()

Note that expected *within* sample error variance is effectively zero!

Now construct $\hat{f}$ and plot predictions:



In [1]:
def fhat(x,alpha):

    try: # Make alpha 1-d for calculations here
        alpha = alpha.squeeze()
    except AttributeError: # Maybe a list?
        pass
    
    yhat = 0
    for k,phik in phis.items():
        yhat += alpha[k]*phik(x)

    return yhat

Domain = np.linspace(0,2*np.pi,100).tolist()

Plot me!



In [1]:
_ = ax.plot(Domain,[fhat(x,alpha) for x in Domain])

### Evaluating Fit



We&rsquo;d like a measure of how well we&rsquo;ve done in estimating the
conditional expectation $f$.  First, compute the mean squared error
(MSE).  Note that when we don&rsquo;t know the true function `f_0` that this
is isn&rsquo;t feasible to compute.  Here we compute a sum of squared
prediction errors as a crude way of computing the integral $$ \int
(f_0(x) - \hat{f}(x))^2dx.  $$ This Riemann integral is appropriate
here because $x$ is uniformly distributed&#x2013;in general we want to
integrate with respect to $dF$.



In [1]:
dx = Domain[1]-Domain[0]
MSE = np.sum([((f0(x) - fhat(x,alpha))**2)*dx for x in Domain])

MSE

Note that the disturbances $u$ don&rsquo;t
enter here&#x2014;this is the mean squared *approximation* error, not the
mean squared error of predictions of actual realizations of the data
$(y,X)$.  If we wanted the latter we&rsquo;d compute
$$
   \mbox{E}((y-\hat{f}(X))^2) =  \mbox{E}((y-f(X)-\epsilon)^2) =
\mbox{E}u^2 + \mbox{MSE}.
$$



#### Leave-one-out estimator



We next estimate $f$ using the &ldquo;leave-one-out&rdquo; approach.



In [1]:
Yminus = []
Alpha = {}
for j in range(N):
    alphaminus = pd.DataFrame(np.linalg.lstsq(TX[TX.index!=j],
                                         y[TX.index!=j],rcond=None)[0],index=TX.columns)
    yminus = lambda x: fhat(x,alphaminus)
    Alpha[j]=alphaminus.squeeze()
    Yminus.append(yminus)

Alpha = pd.DataFrame(Alpha)

yhat = lambda x: np.array([yminus(x) for yminus in Yminus]).mean()

Take a look at $f$ and the leave-one-out estimator:



In [1]:
fig, ax = plt.subplots()

ax.scatter(X,y,label='Data')

ax.plot(Domain,[f0(x) for x in Domain],label='$f_0$')

ax.plot(Domain,[fhat(x,alpha) for x in Domain],label='$\hat{f}$')

ax.plot(Domain,[yhat(x) for x in Domain],label='$\hat{y}_{-}$')
ax.legend()

In [1]:
fig, ax = plt.subplots()

ax.plot(Domain,[yhat(x) - fhat(x,alpha) for x in Domain],label='$\hat{y}(x) - \hat{f}(x)$')

#### Cross-validation error



The cross-validation criterion relies on the leave-one-out estimator
      $$
          \mbox{CV} = \frac{1}{N}\sum_{j=1}^N e_{-j}^2.
      $$



In [1]:
CV = 0
for j in range(N):
     CV += (y.squeeze()[j] - Yminus[j](X[j]))**2/N

## Zeros in expenditure data



### Introduction



Consider the following data from Uganda, collected at the household
  level.  The data itself is *recall* data; the respondent is asked to
  recall the value, the quantity, and the price of consumption out of
  expenditures over the past week, for a rather long list of possible
  non-durable expenditure items.  I&rsquo;ve organized the data as an array,
  with each row corresponding to a household, and each column
  corresponding to a different consumption item.



In [1]:
import pandas as pd

x = pd.read_parquet('uganda_expenditures.parquet')

One thing to note about these data is the large number of &ldquo;zeros&rdquo;.
  This may reflect the fact that few households consume all different
  kinds of consumption goods every week, or could reflect &ldquo;missing&rdquo;
  data on non-zero expenditures (e.g., if the respondent forgot).



In [1]:
# Count of non-missing observations by year (t) and market (mkt)
x.groupby(['t','mkt']).count().T

Missing data can cause serious problems in a demand analysis,
   depending on how and why data might be missing.  If observations
   are &ldquo;missing at random&rdquo; (MAR) then it may be an easy issue to
   address, but if the probability of being missing is related to the
   disturbance term in the demand equation this becomes a sort of
   selection problem that will complicate estimation and inference.



### Household characteristics



One class of variables that may help to explain zeros are
   &ldquo;household characteristics&rdquo;; this includes household size and
   composition (both because this affects demand and perhaps because
   there are more potential shoppers); whether a household is urban or
   rural, and perhaps other characteristics.

Here are some characteristics for the households in Uganda:



In [1]:
z = pd.read_parquet('uganda_hh_characteristics.parquet')
z

### Data mining



Unfortunately, demand theory doesn&rsquo;t offer much guidance to let us
   know how household characteristics should be related to the
   probability of a goods&rsquo; consumption being positive in a given week;
   this is a case where a certain amount of &ldquo;data mining&rdquo; may be a
   reasonable approach.

We&rsquo;ll use tools we&rsquo;ve discussed in class, relying on an
implementation given by the `scikit.learn` project.  In the first
instance, let&rsquo;s consider simply estimating a logit, where the
dependent variable is simply a dummy indicating that the
expenditure of a given good $i$ for a household $j$ at time $t$ is
positive, and where the right-hand-side variables are all the
household characteristics in `z`, combined with a collection of
time dummies (which we can think of as picking up the influence of
prices, among other things):



In [1]:
from sklearn.linear_model import LogisticRegression

time_effects = pd.get_dummies(z.reset_index()[['t']].set_index(z.index),columns=['t'])

X = pd.concat([z,time_effects],axis=1).dropna(how='any') # Drop missing data
x = x.dropna(how='all',axis=1)

# Here's a good place to limit the number of dependent variables
# if we want to save time.  We select just the first few columns:
x = x.iloc[:,:5]

Ests = {}
for item in x: # Iterate over dummies indicating positive expenditure
    y = (x>0)[item]  # Dummy for non-missing item expenditures
    Ests[item] = LogisticRegression(fit_intercept=False,penalty='none').fit(X,y)

#### Coefficients



This gives us a vector of coefficients for each good, which we can
re-arrange into a pandas DataFrame.  Recall that in the logit model
$e^{X\beta}$ is interpreted as the *odds*.  Thus, for a variable in
$X$ which is itself a logarithm, like log HSize, the associated
coefficient can be interpreted as an elasticity.  Accordingly, if the
coefficient on log HSize in the regression involving Matoke is 0.6,
then we can say that for every one percent increase in household size
(other things equal) there&rsquo;s roughly a 0.6% increase in the odds of
observing positive Matoke consumption.  

Coefficients associated with variables in levels have the
interpetation of *semi-elasticities*; thus, the odds of a rural
household consuming Matoke are approximately 53% less than that for
the average household in the sample.  What is the interpretation of
the coefficients associated with discrete counts of different
household members?



In [1]:
Coefs = pd.DataFrame({i:Ests[i].coef_.squeeze() for i in Ests.keys()},index=X.columns)
Coefs

#### Cross-Validation & Lasso



Interpreting the coefficients above allows us to think about how
differences in household characteristics affect the odds of consuming
a particular good, but our original concern was that the data might
not be *missing at random*, which could complicate subsequent
estimation of a demand system.  

Here we use Lasso & cross-validation to tune the Lasso penalty
parameter to check which (if any) of our regressors is useful for
out-of-sample prediction.  

We again use a canned routine from sklearn, `LogisticRegressionCV`.
This bundles both the Lasso penalty criterion and cross-validation
together for us, and searches over a list of penalty parameters to
minimize the EMSE, computed via $K$-fold cross-validation.



In [1]:
from sklearn.linear_model import LogisticRegressionCV
import numpy as np

Lambdas = np.logspace(-5,5,11)

CVEsts = {}
for item in x: # Iterate over dummies indicating positive expenditure
    print(item)
    y = (x>0)[item]  # Dummy for non-missing item expenditures

    # Use 5-fold cross-validation in computing CV statistics; using
    # penalty 'l1' implies a lasso estimator.
    CVEsts[item] = LogisticRegressionCV(fit_intercept=False,
                                        Cs = 1/Lambdas,        # Penalty 1/lambdas to search over
                                        cv=5,                 # K folds
                                        penalty='l1',         # Lasso penalty
                                        solver='liblinear',
                                        scoring='neg_mean_squared_error', # (minus) our CV statistic
                                        n_jobs=-1             # Number of cores to use (-1=all)
                                       ).fit(X,y)

CVCoefs = pd.DataFrame({i:CVEsts[i].coef_.squeeze() for i in CVEsts.keys()},index=X.columns)
CVCoefs

We can see how the estimated coefficients vary with different choices
of the penalty parameter $\lambda$ ($=1/C$).  Consider just the
coefficients associated with estimation of the Matoke logit: If we try
$P$ different values of the penalty parameter using $K$-fold
cross-validation this will be $KP$ different estimates for every
parameter.  We can average over the $K$ different folds to get a
clearer picture of how coefficients vary with &lambda;



In [1]:
pd.DataFrame(CVEsts['Matoke'].coefs_paths_[True].mean(axis=0),index=Lambdas.tolist(),columns=X.columns).T

and see also how the EMSE varies with $\lambda$



In [1]:
EMSEs={k:-e.scores_[True].mean(axis=0).ravel() for k,e in CVEsts.items()} 

EMSEs = pd.DataFrame(EMSEs,index=np.log(Lambdas).tolist()).T
EMSEs

Plotting these versus $\log\lambda$:



In [1]:
EMSEs.T.plot()

Finding the minima of these curves gives estimates of the optimal
&lambda;:



In [1]:
lambda_star = pd.Series({k:1/e.C_[0] for k,e in CVEsts.items()})
lambda_star

Large values of &lambda; encourage parsimony in the selection of
regressors, so it&rsquo;s not surprising to find that consumption items with
large values of $\lambda^*$  also have few regressors (this is the
magic of Lasso):



In [1]:
Lasso_outcomes = pd.DataFrame({'#Regressors':(np.abs(CVCoefs)>1e-5).sum(),
                               'λ*':lambda_star})
Lasso_outcomes

## Neural Networks to Model Effects of Household Composition on Consumption



We&rsquo;ll use `tensorflow` and `keras` to construct a multi-layer neural
  network.  The goal is to use data on household size and composition
  to predict expenditures on different kinds of food.



### Preface



Begin by specifying a place on disk where the model can be stored,
and then loading the modules we&rsquo;ll be using.



In [1]:
# Directory where model & friends is stored...
fn = '/home/ligon/tmp/hh_characteristics_on_food.model/'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cfe
import pickle

### Custom loss functions



In our application there are many observations on food expenditures
which are zero or missing.  We don&rsquo;t think these are very informative,
and don&rsquo;t want our model to try to &ldquo;explain&rdquo; these missing data.  So,
we construct a simple &ldquo;custom&rdquo; loss function which simply zeros out
any loss associated with a missing observation.



In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops


def no_missing_loss(y_true,y_pred,metric='mse'):
    """
    Allow for missing values of y_true so these don't contribute to loss.
    """
    y_pred = ops.convert_to_tensor_v2_with_dispatch(y_pred)
    y_true = math_ops.cast(y_true, y_pred.dtype)    

    miss = tf.cast(y_true==9999,dtype=float)
    notmissing = tf.subtract(1.,miss)
    
    if metric in ['mse','mean_squared_error']:
        losses = tf.square(y_true-y_pred)
    elif metric in ['mae','mean_absolute_error']:
        losses = tf.abs(y_true-y_pred)
    else:
        raise NotImplementedError("Metric %s not implemented." % metric)

    d2 = tf.multiply(losses,notmissing)

    N = tf.reduce_sum(notmissing)
    if N>0:
        return tf.reduce_sum(d2)/N
    else:
        return N

# Define metric "mean absolute error".

def mae(y,yhat): return no_missing_loss(y,yhat,metric='mae')

# Needed to save model later
custom_objects = {'no_missing_loss':no_missing_loss,
                  'mae':mae}

### Get data



Pull in the data we wish to use.



In [1]:
def get_data(LSMS_Library = '~/Data/LSMS_Library/'):

    #################################
    ## Deal with Data
    #################################

    z = pd.read_parquet(LSMS_Library + 'Uganda/_/household_characteristics.parquet')

    x = pd.read_parquet(LSMS_Library + 'Uganda/_/food_expenditures.parquet')

    x = x.fillna(0)

    x,z = cfe.df_utils.drop_missing([x,z])

    x.columns = x.columns.str.replace(' ','_',regex=False)  # No spaces allowed
    x.columns = x.columns.str.replace('(','',regex=False)  # Or parentheses
    x.columns = x.columns.str.replace(')','',regex=False)  # Or parentheses
    x.columns = x.columns.str.replace(',','',regex=False)  # Or commas
    x.columns = x.columns.str.replace('?','',regex=False)  # Or questionmarks

    #x = np.log(x + np.sqrt(x**2+1)) # arcsinh transformation
    x = np.log(x)

    # Replace -inf with finite sentinel value
    x = x.replace(-np.inf,9999)

    #x = keras.utils.normalize(x)

    # Randomly shuffle input data

    x = x.sample(frac=1)
    z = z.loc[x.index]

    # Construct dummies for (t,)
    t = pd.get_dummies(z.reset_index()['t'])
    t.index = z.index

    inputdf = pd.concat([t,z],axis=1)

    outputdf = x

    return inputdf,outputdf

### Construct & Train Model



In [1]:
def construct_model(Foods):
    #################################
    ## Construct model
    #################################

    inputs = keras.Input(shape=(inputdf.shape[1],),
                         dtype=tf.float16,
                         name='hh_characteristics')

    features = layers.Dense(2*128,activation='relu')(inputs)
    residual = features
    #features = layers.Dropout(0.5)(features)
    features = layers.Dense(2*128,activation='relu')(features)
    features = layers.add([features,residual]) # "Skip" block
    features = layers.BatchNormalization()(features)

    features = layers.Dense(2*64,activation='relu')(features)
    residual = features
    #features = layers.Dropout(0.5)(features)
    features = layers.Dense(2*64,activation='relu')(features)
    features = layers.add([features,residual]) # "Skip" block
    features = layers.BatchNormalization()(features)

    features = layers.Dense(2*32,activation='relu')(features)
    residual = features
    #features = layers.Dropout(0.5)(features)
    features = layers.Dense(2*32,activation='relu')(features)
    features = layers.add([features,residual]) # "Skip" block
    features = layers.BatchNormalization()(features)

    features = layers.Dense(2*32,activation='relu')(features)


    # Output layer
    outputs = [layers.Dense(1,name=s)(features) for s in Foods]


    model = keras.Model(inputs = inputs,
                        outputs = outputs)

    model.compile(optimizer="rmsprop",
                  loss = no_missing_loss,
                  metrics = [mae])

    return model

def train(model,inputdf,outputdf):
    #################################
    ## Train model
    #################################

    callbacks_list = [] #[keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5)]


    history = model.fit(x=inputdf,
                        y=outputdf,
                        epochs = 100,
                        validation_split=0.3,
                        batch_size=64,
                        callbacks=callbacks_list)

    return history,model

### Main (for training model to predict log expenditures)



In [1]:
if False: # __name__=='__main__':
    inputdf,outputdf = get_data()

    inputdf.to_parquet(fn+'inputdf.parquet')
    outputdf.to_parquet(fn+'outputdf.parquet')

    # Choose outputs
    Foods = [s for s in outputdf.columns.tolist() if 'Matoke' in s]  # Several categories of matoke
    Foods += ['Infant_Formula']
    Foods = [s for s in outputdf.columns.tolist()]

    model = construct_model(Foods)

    history,model = train(model,inputdf,{k:outputdf[k] for k in Foods})

    model.save(fn)

    with open(fn + 'history.pickle','wb') as f:
        pickle.dump(history.history,f)

### Main (for training model to predict CFE errors)



In [1]:
if __name__=='__main__':
    inputdf,outputdf = get_data()

    # Linear estimation of CFE model
    try:
        r = cfe.from_dataset(fn + 'cfe_result.ds')
    except IOError:
        z = inputdf.filter(regex=' ')
        z['m'] = 'Uganda'

        z = z.reset_index().set_index(['j','t','m'])

        y = outputdf.replace(9999,np.nan)
        y['m'] = 'Uganda'

        y = y.reset_index().set_index(['j','t','m'])

        r = cfe.Result(y=y,z=z)
        r.iterated_estimation()
        r.to_dataset(fn + 'cfe_result.ds')

    # Get Linear CFE residuals
    e = r.e.to_dataframe().squeeze().unstack('i').dropna(how='all').droplevel('m')
    e.index.names = ['j','t']

    e,inputdf = cfe.df_utils.drop_missing([e.fillna(9999),inputdf])    # Choose outputs

    e = e.astype('float32')

    Foods = [s for s in e.columns.tolist()]

    model = construct_model(Foods)

    history,model = train(model,inputdf,{k:e[k] for k in Foods})

    model.save(fn)

    with open(fn + 'history.pickle','wb') as f:
        pickle.dump(history.history,f)

### Evaluate Model



Here&rsquo;s a a function to plot loss or other metrics, with a focus on
comparing validation loss to training loss.



In [1]:
<<preface>>

def plot_validation_metric(history,metric='loss',total=False,log=False):
    h = history
    if not total:
        H = [(k,v) for k,v in h.items() if metric in k]
    else:
        H = [(k,v) for k,v in h.items() if k in [metric,'val_%s' % metric]]

    epochs = range(len(H[0][1]))
    fig, ax = plt.subplots()
    for i in H:
        y = i[1]
        if log: y = np.log(y)
        ax.plot(epochs, y,label=i[0])
        ax.set_xlabel("Epochs")
        if not log:
            ax.set_ylabel(metric)
        else: 
            ax.set_ylabel('log '+metric)

    ax.legend()

    return ax

Here are some functions to recover simple ways to evaluate the
function or its gradients.



In [1]:
def input_gradient(model,x):
    """Gradient of predicted outputs w.r.t. inputs x.
    """

    x = pd.DataFrame(x).T

    myx = tf.Variable(x)

    d = {}
    with tf.GradientTape(persistent=True,watch_accessed_variables=False) as tape:
        tape.watch(myx)
        y = model(myx)

    for j,name in enumerate(model.output_names):
        d[name]=tape.gradient(y[j],myx).numpy().squeeze()

    return pd.DataFrame(d,index=x.columns)

def vary_input(model,x,name,domain):
    """Return model predictions when varying input `name` over domain,
       holding other inputs fixed at x0.
    """
    def my_x(xi,x,name):
        x0 = x.copy().squeeze()
        x0[name] = xi
        return tf.Variable(pd.DataFrame(x0).T)

    y = []
    for v in domain:
        out = model(my_x(v,x,name))
        y.append([foo.numpy() for foo in out])

    return np.array(y).squeeze()

Now, actually load a model and consider some evaluations of it.



In [1]:
# Load model, with custom objects

from train_nn import custom_objects, get_data

inputdf = pd.read_parquet(fn + 'inputdf.parquet')
y = pd.read_parquet(fn + 'outputdf.parquet')

with open(fn + 'history.pickle','rb') as f:
    historyd = pickle.load(f)

model = keras.models.load_model(fn,custom_objects=custom_objects)

# Plot validation graph
plot_validation_metric(historyd,log=True,total=True)

plt.show()

yhat = pd.DataFrame(np.array(model.predict(inputdf)).squeeze().T,columns=model.output_names,index=y.index)

y = y.replace(9999,np.nan)
foo = y.join(yhat,rsuffix='_hat')

#fig,ax = plt.subplots(1,3)

#foo.plot.scatter(x='Passion_Fruits_hat',y='Passion_Fruits',ax=ax[0])
#foo.plot.scatter(x='Onions_hat',y='Onions',ax=ax[1])
#foo.plot.scatter(x='Soda_hat',y='Soda',ax=ax[1])

#plt.show()

### Compare with Linear Regression



In [1]:
import cfe

inputdf,outputdf = get_data()

z = inputdf.filter(regex=' ')
z['m'] = 'Uganda'

z = z.reset_index().set_index(['j','t','m'])

y = outputdf.replace(9999,np.nan)
y['m'] = 'Uganda'

y = y.reset_index().set_index(['j','t','m'])

r = cfe.Result(y=y,z=z)
r.get_beta()

r['loglambdas'] = r['loglambdas']*0

y_lin = r.get_predicted_log_expenditures()
y_lin = y_lin.squeeze().to_dataframe()['yhat'].unstack('i').dropna(how='all')
foo.join(y_lin,rsuffix='_lin')